In [ ]:
# Make module from local packages importable
import os
import sys
os.chdir("/local/home/mhoerold/entrack")

In [ ]:
from src.baum_vagan.vagan.model_wrapper import VAGanWrapper
from src.baum_vagan.utils import map_image_to_intensity_range
from src.baum_vagan.utils import ncc
import yaml
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters

## Define some constants

In [ ]:
smt_label = "20180817-075128"
config_path = os.path.join("data", smt_label, "config.yaml")
model_dir = os.path.join("data", smt_label, "logdir")
with open(config_path, 'r') as f:
    model_config = yaml.load(f)

In [ ]:
model_config["model_id"] = ""
model_config

In [ ]:
wrapper = VAGanWrapper(**model_config)

## Load model

In [ ]:
wrapper.vagan.load_weights(model_dir)

In [ ]:
def four_disks(effect_size=50., image_size=100, moving_effect=True,
               big_rad=15, small_rad=9, big=[True, True, True, True],
               center_fixed=True):
    stdbckg = 50.  # std deviation of the background
    stdkernel = 2.5  # std deviation of the Gaussian smoothing kernel
    img = np.zeros([image_size, image_size])

    def draw_disk(center, radius):
        i_pos = range(center[0] - radius - 1, center[0] + radius + 1)
        j_pos = range(center[1] - radius - 1, center[1] + radius + 1)

        for i in i_pos:
            for j in j_pos:
                # check if inside circle
                if (i - center[0]) ** 2 + (j - center[1]) ** 2 <= radius ** 2:
                    img[i, j] = effect_size

    # Compute disk centers
    offset = 10
    if not center_fixed:
        # Large disks
        c1 = (offset + big_rad, offset + big_rad)
        c2 = (offset + big_rad, image_size - offset - big_rad)
        # Small disks
        c3 = (image_size - offset - big_rad, offset + big_rad)
        c4 = (image_size - offset - big_rad, image_size - offset - big_rad)
    else:
        off = int(image_size / 4)
        c1 = (off, off)
        c2 = (off, image_size - off)
        c3 = (image_size - off, off)
        c4 = (image_size - off, image_size - off)

    centers = [c1, c2, c3, c4]
    for b, c in zip(big, centers):
        if b:
            draw_disk(c, big_rad)
        else:
            draw_disk(c, small_rad)

    noise = np.random.normal(
        scale=stdbckg, size=np.asarray([image_size, image_size])
    )
    smnoise = filters.gaussian(noise, stdkernel)
    smnoise = smnoise / np.std(smnoise) * stdbckg
    img_with_noise = img + smnoise

    return img, img_with_noise

## Test sampler

In [ ]:
sampler_AD = lambda bs: wrapper.data.testAD.next_batch(bs)[0]

In [ ]:
batch = sampler_AD(1)
img = batch[0]
print(img[0, 0, 1])
plt.imshow(img[:, :, 0], cmap='gray')
print(type(img[0, 0, 1]))

In [ ]:
plt.imshow(img[:, :, 2], cmap='bwr', vmin=-1, vmax=1)

## Predict a difference map

In [ ]:
print(batch[0, 0, 0, 1])

fresh, noise = four_disks(
    effect_size=100.0,
    image_size=112,
    big_rad=13,
    big=[True, True, True, True]
)
unknown_disk = map_image_to_intensity_range(noise, -1, 1)
# batch[0, :, :, 1] = wrapper.data.normalize_delta(8)
batch[0, :, :, 0] = unknown_disk
masks = wrapper.vagan.predict_mask(batch)
mask = masks[0]
mask.shape
print(batch.shape)
plt.figure()
plt.imshow(unknown_disk, cmap='gray')

In [ ]:
plt.imshow(mask[:, :, 0], cmap='bwr', vmin=-1, vmax=1)

In [ ]:
plt.imshow(img[:, :, 0] + mask[:, :, 0], cmap='gray', vmin=-1, vmax=1)

In [ ]:
plt.imshow(img[:, :, 0], cmap='gray', vmin=-1, vmax=1)

In [ ]:
print(np.min(img[:, :, 0]))
print(np.max(img[:, :, 0]))

## Unknown t0

In [ ]:
import math
new_radii = list(range(8, 25))
known = [10, 15, 17, 20]
known_delta = 6

masks = []
inps = []
for rad in new_radii:
    img, img_with_noise = four_disks(
        effect_size=100.0,
        image_size=112,
        big_rad=rad,
        big=[True, True, True, True]
    )
    inp = map_image_to_intensity_range(img_with_noise, -1, 1)
    inp = np.reshape(inp, (112, 112, 1))
    inp = np.concatenate((inp, inp * 0 + known_delta, inp), axis = -1)
    inps.append(inp)
    
batch = np.array(inps)
masks = wrapper.vagan.predict_mask(batch)
masks = np.squeeze(masks)

plt.figure(figsize=(14,14))

nrows = int(math.ceil(np.sqrt(batch.shape[0])))
ncols = nrows
for i, mask in enumerate(masks):
    plt.subplot(nrows, ncols, i + 1)
    plt.imshow(mask, cmap='bwr', vmin=-1, vmax=1)
    plt.axis('off')
    radius = new_radii[i]
    plt.title("Radius={}, known={}".format(radius, radius in known))

## Unknown delta

In [ ]:
new_deltas = list(range(3, 10))
known = [4, 6, 8]
known_delta = 6

masks = []
inps = []
for delta in new_deltas:
    img, img_with_noise = four_disks(
        effect_size=100.0,
        image_size=112,
        big_rad=15,
        big=[True, True, True, True]
    )
    inp = map_image_to_intensity_range(img_with_noise, -1, 1)
    inp = np.reshape(inp, (112, 112, 1))
    inp = np.concatenate((inp, inp * 0 + delta, inp), axis = -1)
    inps.append(inp)
    
batch = np.array(inps)
masks = wrapper.vagan.predict_mask(batch)
masks = np.squeeze(masks)

plt.figure(figsize=(14,14))

nrows = int(math.ceil(np.sqrt(batch.shape[0])))
ncols = nrows
for i, mask in enumerate(masks):
    plt.subplot(nrows, ncols, i + 1)
    plt.imshow(mask, cmap='bwr', vmin=-1, vmax=1)
    plt.axis('off')
    delta = new_deltas[i]
    plt.title("Delta={}, known={}".format(delta, delta in known))

In [ ]:
len(wrapper.data.testAD.ids)

## Test NCC

In [ ]:
sampler = lambda bs: wrapper.data.testAD.next_batch(bs)[0]
scores = []
for _ in wrapper.data.testAD.ids:
    batch = sampler(1)
    #print(batch.shape)
    #plt.imshow(batch[0, :, :, 2])
    gt = batch[0, :, :, 2]
    masks = wrapper.vagan.predict_mask(batch)
    pred = masks[0, :, :, 0]
    #print(masks.shape)
    #plt.figure()
    #plt.imshow(np.squeeze(masks))
    sc = ncc(gt, pred)
    scores.append(sc)
    
print("mean {}".format(np.mean(scores)))
print("median {}".format(np.median(scores)))
print("std {}".format(np.std(scores)))

In [ ]:
len(scores)